In [ ]:
import numpy as np # math library, simulation of random variables
import matplotlib.pyplot as plt
import seaborn as sns # more fancy stats visualization
from scipy import stats # stats library
from scipy import optimize # optimization library 
import statsmodels.api as sm # statistical modeling library

In this notebook, we run simulation about a hypothetical clinical trial scenario, in which we are comparing two drugs, with respective (unknown) probability of success $p_1$ and $p_2$. We are interested in testing $\mathcal{H}_0 : (p_1 = p_2)$ against $\mathcal{H}_1 : (p_1 \neq p_2)$ based on the observation of the efficacy of the treatments for $n$ patients who received treatment $1$, $X=(X_1,\dots,X_n)$ and $n$ patients who received treatment $2$, $Y=(Y_1,\dots,Y_n)$. 

All the observations are assumed to be independent with $X_i \sim \mathcal{B}(p_1)$ and $Y_{j} \sim \mathcal{B}(p_2)$. 

We will explore numerically the properties of two tests seen in class: the Wald test and the (Generalized) Likelihood ratio test. 

## Generating Bernoulli data

In [ ]:
## Generating data 
n = 100 # number of patients in each group
p1 = 0.8 # probability of success of treatment 1
p2 = 0.9 # probability of success of treatment 2

rng = np.random.RandomState(100)
X = (rng.random(n)<p1).astype(int)
print("average efficacy of treatment 1 is", np.mean(X))

Y = (rng.random(n)<p2).astype(int)
print("average efficacy of treatment 2 is", np.mean(Y))

# Wald test 

We recall the principle of the Wald test. If our test is about a parameter being equal to a certain value (here $p_1-p_2 = 0$), propose an estimator of this parameter, derive its asymptotic distribution (with a data-dependent variance), and deduce a Gaussian test statistic for your test. In our case, the estimator will be $\hat{p}_{1,n} - \hat{p}_{2,n}$ where $\hat{p}_{1,n} = \frac{1}{n}\sum_{i=1}^{n} X_i$ is the average efficacy for patients who received treatment 1, and $\hat{p}_{2,n} = \frac{1}{n}\sum_{i=1}^{n} Y_i$ is the average efficacy for patients who received treatment 2. 

In Exercise 3, TD3, we proved that, for any $p_1,p_2$, 
$$\frac{\hat{p}_{1,n} - \hat{p}_{2,n} - (p_1 - p_2)}{\sqrt{\frac{\hat{p}_{1,n}(1-\hat{p}_{1,n})}{n} + \frac{ \hat{p}_{2,n}(1- \hat{p}_{2,n})}{n}}}$$ 
converges in distribution to a $\mathcal{N}(0,1)$ random variable. In particular, under $\mathcal{H}_0$ (that is, when $p_1 = p_2$), the test statistic 
$$T_n = \frac{\hat{p}_{1,n} - \hat{p}_{2,n}}{\sqrt{\frac{\hat{p}_{1,n}(1-\hat{p}_{1,n})}{n} + \frac{ \hat{p}_{2,n}(1- \hat{p}_{2,n})}{n}}}$$
is asymptotically a standard Gaussian. The Wald test with threshold $t$ is given by 
$$D(X) = \mathbb{1}(|T_n| > t)$$

**Complete the code below with the threshold that guarantees an asymptotic level of significance equal to alpha.**


In [ ]:
def Wald_test(X,Y,alpha=0.05):
    n1 = len(X)
    n2 = len(Y)
    p1_hat = np.mean(X)
    p2_hat = np.mean(Y)
    var1 = p1_hat*(1-p1_hat)
    var2 = p2_hat*(1-p2_hat)
    if (var1 ==0)&(var2==0):
        # avoid division by zero
        return 0
    else:
        test_statistic = (p1_hat - p2_hat)/np.sqrt(var1/n1 + var2/n2) 
        threshold = ## TO BE COMPLETED 
        decision = 0
        if (np.abs(test_statistic)>threshold):
            decision = 1
        return decision 

### Running the test

The cell below is running the Wald test for given values of $p_1$, $p_2$ and the number of patients $n$ on simulated Bernoulli data. 

**You can try different values of the parameters and different seeds (corresponding to different observations, that can be seen as observations for different groups of patients), and see when the test is rejecting or keeping H0**

In [ ]:
n = 300 # number of patients in each group
p1 = 0.8
p2 = 0.9

rng = np.random.RandomState(42)
X = (rng.random(n)<p1).astype(int)
print("average efficacy of treatment 1 is", np.mean(X))
Y = (rng.random(n)<p2).astype(int)
print("average efficacy of treatment 2 is", np.mean(Y))

print("decision of the Wald test:", Wald_test(X,Y))

### Visualizing the distribution of the test statistic

A good test should use a test statistic whose *distribution* is significantly different from the null to an alternative, so as to make it very unlikely to reject when $\mathcal{H}_0$ is true (small type one error) and very likely to reject when $\mathcal{H}_1$ is true (large power / small type II error).  

Under $\mathcal{H}_1$, that is when $p_1 \neq p_2$, the statistic $T_n$ satisfies 
$$T_n = Z_n + \frac{p_1 - p_2}{\sqrt{\frac{\hat{p}_{1,n}(1-\hat{p}_{1,n})}{n} + \frac{ \hat{p}_{2,n}(1- \hat{p}_{2,n})}{n}}}$$
where $Z_n$ converges in distribution to a $\mathcal{N}(0,1)$, so its distribution is (approximately) 
$$T_n \simeq \mathcal{N}\left( \frac{(p_1 - p_2)\sqrt{n}}{\sqrt{p_1(1-p_1) + (p_2(1-p_2)}}, 1\right)$$
whereas under $\mathcal{H}_0$, 
$$T_n \simeq \mathcal{N}\left(0, 1\right)$$

The code below visualizes the density of the asymptotic distribution of $T_n$ when $p_1=p_2$ and for the point $p_1=0.8,p_2=0.9$ in the alternative as well as the boundaries of the rejection region (which is outside the two vertical lines).  

**Where do we read the power (at this specific alterantive) on the plot? You can play with the parameters and see how the power evolves with the sample size, the gap between p1 and p2 and the type I error alpha**

In [ ]:
p1=0.8
p2=0.9

n = 200
alpha = 0.05

mean_h1 = np.sqrt(n)*(p1-p2)/(np.sqrt(p1*(1-p1)+p2*(1-p2)))

law1=stats.norm
law2=stats.norm(mean_h1,1)

thres = law1.ppf(1-alpha/2)

xplot = np.linspace(-np.abs(mean_h1)-3,np.abs(mean_h1)+3,num=200)
plt.plot(xplot,law1.pdf(xplot),label="$T_n$ under H0")
plt.plot(xplot,law2.pdf(xplot),label="$T_n$ under H1")
plt.axvline(thres, 0, 1, c='black', linewidth = 1)
plt.axvline(-thres, 0, 1, c='black', linewidth = 1)
plt.legend()
plt.show()

# power computation 
beta = law2.cdf(-thres)+1-law2.cdf(thres)
print("(asymptotic) power at this alternative is",beta)

### Approximation of the test characteristics for a finite sample size 

The above reasonning is purely asymptotic and uses some approximations, but we can use some simulation to actually *estimate* the type I error and power. Indeed, for $p_1=p_2=p$, 
$$\alpha((p,p)) = \mathbb{P}_{(p,p)} (D_n(X,Y) = 1) = \mathbb{E}_{(p,p)}[D_n(X,Y)]$$
which can be estimated using an empirical average. 

Given the simulation of $M$ experiments, giving the observations $(X^{m},Y^{m})$ for $m=1,\dots,M$, we can propose the estimate 
$$\widehat{a}_M = \frac{1}{M}\sum_{m=1}^{M} D_n(X^{m},Y^{m})\;.$$
That is, we run $M$ times the test on $M$ different dataset of patients, and report the average number of rejections. 

In [ ]:
p=0.8 # common value of p1 and p2 for an element in H0
alpha_0 = 0.05 # desired type I error 

M = 1000 # number of simulations 
n_values = np.array([30,50,100,200,300,500,1000]) # number of patients for which we want to estimate the type I error  
n_max =np.max(n_values)
n_nb = len(n_values)

X_table = (np.random.random(size=(M,n_max))<p).astype(int)
Y_table = (np.random.random(size=(M,n_max))<p).astype(int)

alphas = np.zeros(n_nb)

for i in range(n_nb):
    n = n_values[i] 
    for m in range(M):
        X_small = X_table[m,range(n)]
        Y_small = Y_table[m,range(n)]
        decision=Wald_test(X_small,Y_small,alpha_0)
        alphas[i]=alphas[i]+decision
    alphas[i]=alphas[i]/M

plt.plot(n_values,alphas)
plt.axhline(alpha_0,0, 1, c='black', linewidth = 1)
plt.title("empirical type I error at p1=p2={} as a function of the sample size".format(p))
plt.show()


We can do similar simulations for the power, estimated as 
$$\frac{1}{M}\sum_{m=1}^{M} D_n(X^{m},Y^{m})\;.$$
(fraction of rejections) but when the data is simulated under an alternative. 

**How many samples seem necessary to get a power 0.8 under the alternative p1=0.8, p2=0.9?**

In [ ]:
p1=0.8 #specifying the element of H1
p2=0.9 
alpha_0 = 0.05 # desired type I error 

M = 1000 # number of simulations 
n_values = np.array([30,50,100,200,300,500,1000]) # number of patients  
n_max =np.max(n_values)
n_nb = len(n_values)

X_table = (np.random.random(size=(M,n_max))<p1).astype(int)
Y_table = (np.random.random(size=(M,n_max))<p2).astype(int)

powers = np.zeros(n_nb)

for i in range(n_nb):
    n = n_values[i] 
    for m in range(M):
        X_small = X_table[m,range(n)]
        Y_small = Y_table[m,range(n)]
        decision=Wald_test(X_small,Y_small,alpha_0)
        powers[i]=powers[i]+decision
    powers[i]=powers[i]/M

plt.plot(n_values,powers)
plt.axhline(0.8,0, 1, c='red', linewidth = 1)
plt.title("empirical power at p1={},p2={} as a function of the sample size".format(p1,p2))
plt.show()

# Likelihood-Ratio Test

For Bernoulli data, the log-likelihood ratio statistic associated to the test 
$\mathcal{H}_0 : (p_1 = p_2)$ against $\mathcal{H}_1 : (p1\neq 2)$ is 

$$\log \widetilde{\Lambda}(X,Y) = \log\frac{\sup_{p_1 \in [0,1]} p_1^{S_1}(1-p_1)^{n-S_1}\sup_{p_2 \in [0,1]} p_2^{S_2}(1-p_2)^{n-S_2}}{\sup_{p \in [0,1]} p^{S_1}(1-p)^{n-S_1}p^{S_2}(1-p)^{n-S_2}} $$
where $S_1$ is the number of successes of treatment 1 among the $n$ patients who received it and $S_2$ is the number of successes of treatment 2 among the $n$ patients who received it.

Introducing the estimates $\hat{p}_{1,n}$, $\hat{p}_{2,n}$ and $\widehat{p}_{n} = \frac{1}{2n}\sum_{i=1}^{n}(X_i+Y_i)$ (the average efficacy across all patients), we have (dropping the index $n$ for readability)

\begin{align}
\log \widetilde{\Lambda}(X,Y) & = \log\frac{\hat{p}_1^{S_1}(1-\hat{p}_1)^{n-S_1}\hat{p}_2^{S_2}(1-\hat{p}_2)^{n-S_2}}{\hat{p}^{S_1}(1-\hat{p})^{n-S_1}\hat{p}^{S_2}(1-\hat{p})^{n-S_2}} \\
& = n \left(\hat{p}_1 \log \frac{\hat{p}_1}{\hat{p}} + (1-\hat{p}_1) \log \frac{1-\hat{p}_1}{1-\hat{p}}\right) + n \left(\hat{p}_2 \log \frac{\hat{p}_2}{\hat{p}} + (1-\hat{p}_2) \log \frac{1-\hat{p}_2}{1-\hat{p}}\right) \\
& = n \mathrm{kl}(\hat{p}_1,\hat{p}) + n \mathrm{kl}(\hat{p}_2,\hat{p})
\end{align}

where we introduce the KL divergence between two Bernoulli distributions with mean $p$ and $q$ as $\mathrm{kl}(p,q) = p\log\frac{p}{q} + (1-p)\log \frac{1-p}{1-q}$

The functions given below permit to compute the likelihood ratio. 

In [1]:
eps = 1e-15    

def kl(p, q):
    """Kullback-Leibler divergence for Bernoulli distributions."""
    p = min(max(p, eps), 1-eps)
    q = min(max(q, eps), 1-eps)
    return p*np.log(p/q) + (1-p)*np.log((1-p)/(1-q))

def Log_Likelihood_Ratio(X,Y): 
    p1_hat = np.mean(X)
    p2_hat = np.mean(Y)
    p_hat = (p1_hat+p2_hat)/2
    n1 = len(X)
    n2 = len(Y)
    return n1*kl(p1_hat,p_hat)+n2*kl(p2_hat,p_hat)

A LRT test is of the form 
$$D(X,Y) = \mathbb{1}\left(\log\widetilde{\Lambda}(X,Y) > t\right)$$
for some threshold $t$. 

**Complete the function below to implement a LRT guaranteed to have an asymptotic type one error of $\alpha$ using Wilk's theorem.**

In [ ]:
def LRT_test(X,Y,alpha=0.05):
    test_statistic = ## TO BE COMPLETED
    threshold = ## TO BE COMPLETED
    decision = 0
    if (test_statistic>threshold):
        decision = 1
    return decision 

### Running the test 

As before, you can run the test with varying parameters, and also compare its outcome to the Wald test.

In [ ]:
n = 350 # number of patients in each group
p1 = 0.8
p2 = 0.9

rng = np.random.RandomState(42)
X = (rng.random(n)<p1).astype(int)
print("average efficacy of treatment 1 is", np.mean(X))
Y = (rng.random(n)<p2).astype(int)
print("average efficacy of treatment 2 is", np.mean(Y))

print("decision of the LRT test:", LRT_test(X,Y))
print("decision of the Wald test:", Wald_test(X,Y))

### Estimation of the test characteristics

Below, we estimate empirically the type I error of the two tests using simulations. 

In [ ]:
p=0.8 # common value of p1 and p2 for an element in H0
alpha_0 = 0.05 # desired type I error 

M = 100 # number of simulations 
n_values = np.array([30,50,100,200,300,500]) # number of patients   
n_max =np.max(n_values)
n_nb = len(n_values)

X_table = (np.random.random(size=(M,n_max))<p).astype(int)
Y_table = (np.random.random(size=(M,n_max))<p).astype(int)

alphas_LRT = np.zeros(n_nb)
alphas_Wald = np.zeros(n_nb)

for i in range(n_nb):
    n = n_values[i] 
    for m in range(M):
        X_small = X_table[m,range(n)]
        Y_small = Y_table[m,range(n)]
        decision=Wald_test(X_small,Y_small,alpha_0)
        alphas_Wald[i]=alphas_Wald[i]+decision
        decision=LRT_test(X_small,Y_small,alpha_0)
        alphas_LRT[i]=alphas_LRT[i]+decision
    alphas_Wald[i]=alphas_Wald[i]/M
    alphas_LRT[i]=alphas_LRT[i]/M

plt.plot(n_values,alphas_Wald,label="Wald test")
plt.plot(n_values,alphas_LRT,label="LRT test")
plt.axhline(alpha_0,0, 1, c='black', linewidth = 1)
plt.title("empirical type I error at p1=p2={} as a function of the sample size".format(p))
plt.legend()
plt.show()


And we can do the same for the power. 

In [ ]:
p1=0.8 #specifying the element of H1
p2=0.9 
alpha_0 = 0.05 # desired type I error 

M = 100 # number of simulations 
n_values = np.array([30,50,100,200,300,400,500]) # number of patients  
n_max =np.max(n_values)
n_nb = len(n_values)

X_table = (np.random.random(size=(M,n_max))<p1).astype(int)
Y_table = (np.random.random(size=(M,n_max))<p2).astype(int)

powers_Wald = np.zeros(n_nb)
powers_LRT = np.zeros(n_nb)

for i in range(n_nb):
    n = n_values[i] 
    for m in range(M):
        X_small = X_table[m,range(n)]
        Y_small = Y_table[m,range(n)]
        decision=Wald_test(X_small,Y_small,alpha_0)
        powers_Wald[i]=powers_Wald[i]+decision
        decision=LRT_test(X_small,Y_small,alpha_0)
        powers_LRT[i]=powers_LRT[i]+decision
    powers_Wald[i]=powers_Wald[i]/M
    powers_LRT[i]=powers_LRT[i]/M

plt.plot(n_values,powers_Wald,label="Wald test")
plt.plot(n_values,powers_LRT,label="LRT test")
plt.axhline(0.8,0, 1, c='red', linewidth = 1)
plt.title("empirical power at p1={},p2={} as a function of the sample size".format(p1,p2))
plt.legend()
plt.show()

**Explore different values of the parameters p, p1, p2 to investigate potential difference in behavior between the two tests.**

### Distribution of the test statistic

For the LRT, we know the asymptotic distribution of the test statistic under $\mathcal{H}_0$. However, due to its complex form, approximating its asymptotic distribution under $\mathcal{H}_1$ is complicated. In order to compare the distribution of the test statistic uner $\mathcal{H}_0$ and $\mathcal{H}_1$ it remains the possibility to resort (again) to simulation, and report (a histogram of) its empirical distribution under some alternative in $\mathcal{H}_1$. 